# 作業工程計画

使用データ：FMT修正版_CRLEAデータ①入力様式_v2_1222.xlsx<br>
目的：機械学習に直ぐ使用できる形式に変換する<br>

0. ライブラリを読み込む
1. 受領データを読み込む
2. 受領データを整形する
3. 整形データを出力する

## ライブラリ

In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import japanize_matplotlib
# import seaborn as sns
import re 

"""
機械学習ライブラリの準備
"""

from sklearn.model_selection import cross_val_score
# from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.tree import DecisionTreeRegressor #決定木
import lightgbm as lgb #lightGBM

from sklearn import tree
import graphviz

### 村上さん pandas_tool

In [2]:
"""
村上さんtoolbox
"""
# pandas 基礎集計クラス
import numpy as np
import pandas as pd
import copy
import seaborn as sns
from itertools import combinations

#http://qiita.com/tanemaki/items/2ed05e258ef4c9e6caac

# Jupyterで表示するためには、最初に以下を実行すること
%matplotlib inline

# Static Classで設計する
class pandas_tool:
    
    # All in one チェック（Jupyterのみ）
    def all_basic_summary_jupyter(df):
        print("■ 型の確認")
        display(pandas_tool.type(df))
        print("■ 数値型の確認")
        display(pandas_tool.summary(df)[0])
        print("■ カテゴリ型の確認")
        cate_var_data = list(df.select_dtypes(include=['object']).columns)
        ret = pandas_tool.freq(df,cate_var_data)
        for d in ret:
            display(pd.DataFrame(d))
            print("---------------")
        print("■ 欠損の確認")
        display(pandas_tool.check_missing(df))
    
    # 相関関係可視化（Jupyterのみ）
    def all_value_relation_visualize(df):
        #sns.set_context("poster", 1.2, {"lines.linewidth": 3})
        sns.pairplot(df,size=5)
    
    # カテゴリ変数でのヒートマップ（Jupyterのみ）
    def make_heatmap(df,x,y,value):
        target_df = df.pivot_table(index=x,values=value,columns=y)
        sns.heatmap(target_df, annot=True, fmt='1.1f', cmap='Blues')
    
    # 散布図（Jupyterのみ）
    def make_scatter_chart(df,x,y):
        #sns.jointplot(x=x, y=y, data=df, kind="hex")
        sns.jointplot(x=x, y=y, data=df)
    
    # 組み合わせでヒートマップを作成（Jupyterのみ）
    def all_make_heatmap(df,var_list,value):
        col_num = 2
        var_list_set = list(combinations(var_list,2))
        
        fig, axes = plt.subplots(int(len(var_list_set)/col_num)+1, col_num, figsize=(18,3+6.5*int(len(var_list_set)/col_num)))
        
        for i,target in enumerate(var_list_set):
            target_df = df.pivot_table(index=target[0],values=value,columns=target[1])
            sns.heatmap(target_df, annot=True, fmt='1.1f', cmap='Blues', ax=axes[int(i/col_num), i%col_num])
            
        plt.tight_layout()
    
    # 数値集計
    def summary(df,view=False):
        ret=df.describe()
        mis_ret=df.isnull().sum()
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print("・統計量")
            print(ret)
            print("・欠損値")
            print(mis_ret)
            pd.set_option("display.max_columns",param)
        return ret,mis_ret
    
    # 型チェック
    def type(df,view=False):
        ret = df.dtypes
        if view:
            param=pd.get_option("display.max_rows")
            pd.set_option("display.max_rows",1000)
            print(ret)
            pd.set_option("display.max_rows",param)
        return ret
    
    # 欠損チェック
    def check_missing(df,view=False):
        not_null_df=df.notnull()
        ret=pd.DataFrame()
        for name in not_null_df.columns:
            tmp_df=not_null_df[name].value_counts()
            tmp_df.name=name
            ret = pd.concat([ret,tmp_df],axis=1)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 欠損値のオブザベーションを抽出
    def get_miss_data(df,column,view=False):
        ret=df[df[column].isnull()]
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        return ret
    
    # 欠損値を中央値で補完
    def fill_miss_med(df,var_name):
        var=df[var_name].median()
        df[var_name].fillna(var,inplace=True)
        return df
    
    # 欠損値を0で補完
    def fill_miss_zero(df,var_name):
        df[var_name].fillna(0,inplace=True)
        return df
    
    # 特定の値を欠損とみなす
    def apply_miss_value(df,var_name,value):
        df[var_name]=df[var_name].replace(value,np.nan)
        return df
    
    # 重複チェック
    def check_dup(df,columns,view=False):
        ret=pd.DataFrame()
        for name in columns:
            dup_cnt=df[name].duplicated().sum()
            tmp_df = pd.DataFrame({'var_name':[name],'dup_cnt':[dup_cnt]})
            ret = pd.concat([ret,tmp_df],axis=0,ignore_index= True)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 組み合わせ重複チェック
    def check_dup_comb(df,columns,view=False):
        ret = df[columns].duplicated().sum()
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # ユニークデータ取得
    def get_uniq_data(df,uniq_key,sort_key,keep='first'):
        ret = df.sort_values(by=sort_key)
        ret.drop_duplicates(subset=uniq_key, keep=keep, inplace=True)
        return ret
    
    # カテゴリ集計
    def freq(df,columns,view=False):
        ret=list()
        for name in columns:
            tmp_df=df[name].value_counts()
            tmp_df.name=name
            #ret = pd.concat([ret,tmp_df],axis=1)
            ret.append(tmp_df)
        
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            for r in ret:
                print(r)
                #display(r)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # 複雑な集計
    def tabulate(df,row,col=None,var='',func=np.sum,view=False):
        if var == '':
            tmp_df=df.reset_index(drop=False,inplace=False)
            ret=pd.pivot_table(data=tmp_df, values='index', index=row, columns=col, aggfunc='count', dropna=False, fill_value=0 ,margins = False)
            tmp_df=None
        else:
            ret=pd.pivot_table(data=df, values=var, index=row, columns=col, aggfunc=func, dropna=False, fill_value=0 ,margins = False)
        if view:
            param=pd.get_option("display.max_columns")
            pd.set_option("display.max_columns",1000)
            print(ret)
            pd.set_option("display.max_columns",param)
        
        return ret
    
    # マージ
    def merge(df1,df2,key,how,view=True):
        if view:
            print("df1のキー重複")
            pandas_tool.check_dup_comb(df1,key,True)
            print("df2のキー重複")
            pandas_tool.check_dup_comb(df2,key,True)
            
            print("df1のオブザベーション:{0}".format(len(df1)))
            print("df2のオブザベーション:{0}".format(len(df2)))
        
        ret=pd.merge(df1,df2,how=how,on=key)
        
        if view:
            print("mergeのオブザベーション:{0}".format(len(ret)))
        
        return ret
    
    # Rank
    def rank(df,var,num,suffix='_rank',check=False):
        labels=[i for i in range(0,num)]
        df[var+suffix]=pd.qcut(df[var], num, labels=labels)
        
        # check data
        if check:
            ret=pd.DataFrame()
            max_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func=np.max,view=False)
            max_df.name='max'
            min_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func=np.min,view=False)
            min_df.name='min'
            cnt_df=pandas_tool.tabulate(df=df,row=[var+suffix],var=var,func='count',view=False)
            cnt_df.name='count'
            ret=pd.concat([ret,min_df,max_df,cnt_df],axis=1)
            return df,ret
            
        return df
    
    # Rank適用(min基準)
    def apply_rank(df,rank_df):
        tmp_df=copy.deepcopy(rank_df)
        tmp_df.reset_index(drop=False,inplace=True)
        target_name=tmp_df.columns[3]
        tmp_df.columns=["rank","min","max","cnt"]
        
        def judge_thld(row):
            ret_var = -1
            cond_list = ["if 0 : ret_var = 0"]
            
            for i in range(1,len(tmp_df)):
                cond_list.append("elif row < " +str(tmp_df.ix[i,'min'])+ " : ret_var = " + str(tmp_df.ix[i-1,'rank']))
            
            cond_list.append("else: ret_var = " + str(tmp_df.ix[len(tmp_df)-1,'rank']))
            cond_str="\r\n".join(cond_list)
            # ローカル辞書をexecと共有する
            local_dict=locals()
            exec(cond_str,local_dict)
            return local_dict["ret_var"]
        
        df[target_name+"_rank"]=df[target_name].apply(judge_thld)
        return df
    
    # Min%以下はMin%点に、Max%以上はMax%点にクリップする
    def clip_min_max(df,col_list,apply_df=None,max_pct=0.99,min_pct=0.01):
        p_min = df[col_list].quantile(min_pct)
        p_max = df[col_list].quantile(max_pct)
        
        df[col] = df[col_list].clip(p_min,p_max,axis=1)
        
        # もしも適用先のデータがあるならば（例えば検証データ）対応
        if apply_df is not None:
            apply_df[col] = apply_df[col_list].clip(p_min,p_max,axis=1)
            return df,apply_df
        else:
            return df
    
    
    # 文字列→数値変換
    def conv_float(df,column,percent_flg=False):
        
        def conv_f(row):
            if row[column] == "" or row[column] is np.nan:
                return np.nan
            else:
                return float(row[column])
        
        df[column]=df[column].str.replace("\\","").str.replace(",","").str.replace("%","").str.strip()
        df[column]=df.apply(conv_f,axis=1)
        
        if percent_flg:
            df[column]=df[column]/100
        
        return df

## データの読み込み

In [3]:
#エクセルを読み込み、シートごとにデータフレームに入れる
"""
path_name = ファイルパス
sn_list = エクセルシート名のリスト
"""
path_name = "/Users/s.ogura/Documents/CRLEA/data/rawdata/FMT修正版_CRLEAデータ①入力様式_v2_1222.xlsx"
path_file =  r'{}'.format(path_name)
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names
#sn_list ['データ入力様式_英語', 'データ入力様式_国語', 'データ入力様式_数学', '空欄埋め方', 'データ入力様式(記入例)']

# Excelファイルの読み込み
df_eng = pd.read_excel(path_file, sn_list[0], header=0, index_col=None,skiprows=[1],dtype = {'ロット番号': object})#headerを修正
df_jpn = pd.read_excel(path_file, sn_list[1], header=0, index_col=None,skiprows=[1],dtype = {'ロット番号': object})#headerを修正
df_math = pd.read_excel(path_file, sn_list[2], header=0, index_col=None,skiprows=None,dtype = {'ロット番号': object})#headerを修正

In [15]:
path_name = "/Users/s.ogura/Documents/CRLEA/data/rawdata/分野区分マスタv4.1.xlsx"
path_file =  r'{}'.format(path_name)
input_file = pd.ExcelFile(path_file)
sn_list = input_file.sheet_names

#夏採点を読み込んで、採点回を列に追加する　df_summer

bunya = []
bunya = pd.read_excel(path_file, sn_list[0], header=0, index_col=None,skiprows=None)#headerを修正
bunya = bunya.drop(['科目コード','ポイント採点', '解答言語'], axis=1)
#bunya = bunya.fillna({'解答言語': '日本語'})
bunya = bunya.rename(columns={'分野名':'分野修正前'})
bunya.head()

,分野修正前,分野
0,漢文（現代語訳）,漢文(現訳)
1,漢文（内容説明）,漢文(内説)
2,現代文・小説,小説
3,現代文・評論,評論
4,現代文・評論（理由説明）,評論


# 変数追加

In [5]:
def make_rank(df,x=str):# 在宅勤務のフラグを作るために使用。
    # 最終ペースをランク分け
    if x == "最終ペース":
        conditions = [
        (df[x] >= 600),
        (df[x] >= 500),
        (df[x] >= 400),
        (df[x] >= 300),
        (df[x] >= 200),
        (df[x] >= 100),
        (df[x] >= 0)
         ]
        
        choices = ["600~", "500~600", "400~500", "300~400", "200~300", "100~200","0~100"]
        df.loc[:,'最終ランク'] = np.select(conditions, choices, default = 0)
    # 在宅勤務のフラグ作成
    if x == "年度":
        conditions = [
        (df[x] >= 2020),
        (df[x] >= 0)
         ]
        
        choices = [1, 0]
        df.loc[:,'在宅勤務'] = np.select(conditions, choices, default = 0)
    return df

## 国語科目の前処理 

In [6]:
df = df_jpn.copy()
#グレーアウトした行を削除
df = df[df['採点回']!='春']
df = df[df['ロット番号']!='基準なし']
#len(df) 346->307

#列の欠損値を補完
df['必須条件の有無'] = df['必須条件の有無'].fillna(0)#必須条件の有無の空白を0に
df['配点要素の数'] = df['配点要素の数'].fillna(df['ポイント数'])#配点要素の数の空白をポイント数と同じ数に
#最終ペースでランク分け、年度で在宅勤務のフラグ
#lst=['最終ペース','年度']
#for i in lst:
#    df = make_rank(df,i)
df = make_rank(df,'年度')

#分野の表記揺れを修正
df['分野'] = df['分野'].replace('古文（現代語訳）','古文（現訳）')
df['分野'] = df['分野'].replace('古文（内容説明）','古文（内説）')
df['分野'] = df['分野'].replace('漢文（現代語訳）','漢文（現訳）')
df['分野'] = df['分野'].replace('漢文（内容説明）','漢文（内説）')
df['最終ペース'] = df['最終ペース'].replace('採点回終了時',np.nan)

#列のデータタイプを確認
#pandas_tool.type(df_jpn,view=False)
#列のデータタイプを変更
#分野名を分野マスタで変換
#必要な列のみを抽出
df = df[['年度', '採点回', '学年', '科目', '分野','ロット番号', 'ポイント採点', '文字数',
       'ポイント数', '配点要素の数', '配点',  '必須条件の有無','在宅勤務','企画ペース', '最終ペース']]
df_jpn_pretreated = df.copy()

## 数学科目の前処理 

In [7]:
df_math.columns.values

array(['年度', '採点回', '学年', '科目', 'ロット番号', '分野', '（1）実点・立式点', '（1）カッコ点・立式点',
       '（1）実点・答え点', '（1）カッコ点・答え点', '（1）実点・その他', '（1）カッコ点・その他',
       '（2）実点・立式点', '（2）カッコ点・立式点', '（2）実点・答え点', '（2）カッコ点・答え点',
       '（2）実点・その他', '（2）カッコ点・その他', '（3）実点・立式点', '（3）カッコ点・立式点',
       '（3）実点・答え点', '（3）カッコ点・答え点', '（3）実点・その他', '（3）カッコ点・その他',
       '（1）ポイント数合計', '（2）ポイント数合計', '（3）ポイント数合計', 'ポイント採点', '（1）配点',
       '（2）配点', '（3）配点', '企画ペース', '最終ペース', '模範解答の記述行数（１）', '模範解答の記述行数（２）',
       '模範解答の記述行数（３）', '補足資料の枚数', '分野（詳細）', '他参照有無', 'Unnamed: 39'],
      dtype=object)

In [16]:
df = df_math.copy()
#グレーアウトした行を削除
df = df[~((df['採点回'] == '春')&(df['年度']==2019))]#2019春を除外
df = df[~((df['採点回'] == '夏')&(df['年度']==2018)&(df['学年']!=1))]#2018夏2-3年を除外
#len(df) 477->422

#列の欠損値を補完
df['ポイント採点'] = df['ポイント採点'].fillna(0)
df['（1）実点・立式点'] = df['（1）実点・立式点'].fillna(0)
df['（1）カッコ点・立式点'] = df['（1）カッコ点・立式点'].fillna(0)
df['（1）実点・答え点'] = df['（1）実点・答え点'].fillna(0)
df['（1）カッコ点・答え点'] = df['（1）カッコ点・答え点'].fillna(0)
df['（1）実点・その他'] = df['（1）実点・その他'].fillna(0)
df['（1）カッコ点・その他'] = df['（1）カッコ点・その他'].fillna(0)
df['（1）ポイント数合計'] = df['（1）ポイント数合計'].fillna(0)
df['（1）配点'] = df['（1）配点'].fillna(0)
df['模範解答の記述行数（１）'] = df['模範解答の記述行数（１）'].fillna(0)
df['（2）実点・立式点'] = df['（2）実点・立式点'].fillna(0)
df['（2）カッコ点・立式点'] = df['（2）カッコ点・立式点'].fillna(0)
df['（2）実点・答え点'] = df['（2）実点・答え点'].fillna(0)
df['（2）カッコ点・答え点'] = df['（2）カッコ点・答え点'].fillna(0)
df['（2）実点・その他'] = df['（2）実点・その他'].fillna(0)
df['（2）カッコ点・その他'] = df['（2）カッコ点・その他'].fillna(0)
df['（2）ポイント数合計'] = df['（2）ポイント数合計'].fillna(0)
df['（2）配点'] = df['（2）配点'].fillna(0)
df['模範解答の記述行数（２）'] = df['模範解答の記述行数（２）'].fillna(0)
df['（3）実点・立式点'] = df['（3）実点・立式点'].fillna(0)
df['（3）カッコ点・立式点'] = df['（3）カッコ点・立式点'].fillna(0)
df['（3）実点・答え点'] = df['（3）実点・答え点'].fillna(0)
df['（3）カッコ点・答え点'] = df['（3）カッコ点・答え点'].fillna(0)
df['（3）実点・その他'] = df['（3）実点・その他'].fillna(0)
df['（3）カッコ点・その他'] = df['（3）カッコ点・その他'].fillna(0)
df['（3）ポイント数合計'] = df['（3）ポイント数合計'].fillna(0)
df['（3）配点'] = df['（3）配点'].fillna(0)
df['模範解答の記述行数（３）'] = df['模範解答の記述行数（３）'].fillna(0)


#補足資料の枚数列の例外を修正
df['補足資料の枚数'] = df['補足資料の枚数'].replace('補足なし',0)
df['補足資料の枚数'] = df['補足資料の枚数'].replace('補足基準発行後',np.nan)


df['科目'] = df['科目'].replace('数学X/A\u3000','数学X/A')#0905に修正
#最終ペースを空欄に
df['最終ペース'] = df['最終ペース'].replace('採点回終了時',np.nan)
#在宅勤務フラグ作成
df = make_rank(df,'年度')

#分野名を分野マスタで変換
df = df.rename(columns={'分野':'分野修正前'})
df1 = df
df2 = bunya
df = pandas_tool.merge(df1,df2, key = ["分野修正前"],how = 'left')
#不要な列を削除
df = df.drop(['分野（詳細）', '他参照有無', '分野修正前','Unnamed: 39'], axis=1)
df_math_pretreated = df.copy()

df1のキー重複
499
df2のキー重複
0
df1のオブザベーション:529
df2のオブザベーション:159
mergeのオブザベーション:529


## 英語科目の前処理 

In [9]:
df_eng.columns.values

array(['年度', '採点回', '学年', '科目', 'ロット番号', '大問', '小問', '分野', '解答言語',
       '文字/単語数', '配点', 'ポイント採点', 'ポイント・ランク・パート数', '減点要素の数', '企画ペース',
       '最終ペース', 'パターン数'], dtype=object)

In [10]:
def split_columns(df):
    #文字数に数値が入っている場合、そのまま代入
    if str(df).isdigit() == True: #簡単な例から場合分けする。 文字列か、数値か判定
        return float(df)
    elif '+' in str(df):
        x = str(df).split('+')
        return sum(list(map(float, x)))
    elif '-' in str(df):
        x = str(df).split('-')
        return sum(list(map(float, x)))
    elif '‐' in str(df):
        x = str(df).split('‐')
        return sum(list(map(float, x)))
    else:
        return float(df)

In [11]:
df = df_eng.copy()
#グレーアウトした行を削除
df = df[df['分野'] != '英単語']#英単語を除外
#len(df) 333->321

#合計列の追加
df['配点合計'] = df['配点'].apply(split_columns)
df['文字数合計'] = df['文字/単語数'].apply(split_columns)
df['ポイント数合計'] = df['ポイント・ランク・パート数'].apply(split_columns)
df['減点要素の数合計'] = df['減点要素の数'].apply(split_columns)

#列の欠損値を補完
#df = df.fillna(0)#最終ペースの空白もゼロにしているので、修正
df['パターン数'] = df['パターン数'].fillna(0)
df['減点要素の数合計'] = df['減点要素の数合計'].fillna(0)
df['ポイント採点'] = df['ポイント採点'].fillna(0)
df['ポイント数合計'] = df['ポイント数合計'].fillna(0)

#年度で在宅勤務のフラグ
df = make_rank(df,'年度')

#最終ペースを空欄に
df['最終ペース'] = df['最終ペース'].replace('採点回終了時',np.nan)

#必要な列のみを抽出
df = df.drop(['小問','文字/単語数', '配点', 'ポイント・ランク・パート数', '減点要素の数',], axis=1)
df_eng_pretreated = df.copy()

In [17]:
"""
Excelの書き出し 集計用データマート
"""

name = "FMT修正版_CRLEAデータ①入力様式_v2_1222_pretreated_v1"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス


df1 = df_jpn_pretreated
df2 = df_math_pretreated
df3 = df_eng_pretreated

#csv
df1.to_csv('{}/{}.csv'.format(path_folder,"{}_{}".format(name,'jpn')),encoding='utf-8-sig',index=False)
df2.to_csv('{}/{}.csv'.format(path_folder,"{}_{}".format(name,'math')),encoding='utf-8-sig',index=False)
df3.to_csv('{}/{}.csv'.format(path_folder,"{}_{}".format(name,'eng')),encoding='utf-8-sig',index=False)

#Excel
with pd.ExcelWriter('{}/{}.xlsx'.format(path_folder,name)) as writer:
    df1.to_excel(writer, sheet_name='国語',encoding='utf-8-sig', index = False)
    df2.to_excel(writer, sheet_name='数学',encoding='utf-8-sig', index = False)
    df3.to_excel(writer, sheet_name='英語',encoding='utf-8-sig', index = False)

## 数学の分野修正

In [18]:
df = df_math.copy()
#グレーアウトした行を削除
# df = df[~((df['採点回'] == '春')&(df['年度']==2019))]#2019春を除外
# df = df[~((df['採点回'] == '夏')&(df['年度']==2018)&(df['学年']!=1))]#2018夏2-3年を除外
#len(df) 477->422

#列の欠損値を補完
df['ポイント採点'] = df['ポイント採点'].fillna(0)
df['（1）実点・立式点'] = df['（1）実点・立式点'].fillna(0)
df['（1）カッコ点・立式点'] = df['（1）カッコ点・立式点'].fillna(0)
df['（1）実点・答え点'] = df['（1）実点・答え点'].fillna(0)
df['（1）カッコ点・答え点'] = df['（1）カッコ点・答え点'].fillna(0)
df['（1）実点・その他'] = df['（1）実点・その他'].fillna(0)
df['（1）カッコ点・その他'] = df['（1）カッコ点・その他'].fillna(0)
df['（1）ポイント数合計'] = df['（1）ポイント数合計'].fillna(0)
df['（1）配点'] = df['（1）配点'].fillna(0)
df['模範解答の記述行数（１）'] = df['模範解答の記述行数（１）'].fillna(0)
df['（2）実点・立式点'] = df['（2）実点・立式点'].fillna(0)
df['（2）カッコ点・立式点'] = df['（2）カッコ点・立式点'].fillna(0)
df['（2）実点・答え点'] = df['（2）実点・答え点'].fillna(0)
df['（2）カッコ点・答え点'] = df['（2）カッコ点・答え点'].fillna(0)
df['（2）実点・その他'] = df['（2）実点・その他'].fillna(0)
df['（2）カッコ点・その他'] = df['（2）カッコ点・その他'].fillna(0)
df['（2）ポイント数合計'] = df['（2）ポイント数合計'].fillna(0)
df['（2）配点'] = df['（2）配点'].fillna(0)
df['模範解答の記述行数（２）'] = df['模範解答の記述行数（２）'].fillna(0)
df['（3）実点・立式点'] = df['（3）実点・立式点'].fillna(0)
df['（3）カッコ点・立式点'] = df['（3）カッコ点・立式点'].fillna(0)
df['（3）実点・答え点'] = df['（3）実点・答え点'].fillna(0)
df['（3）カッコ点・答え点'] = df['（3）カッコ点・答え点'].fillna(0)
df['（3）実点・その他'] = df['（3）実点・その他'].fillna(0)
df['（3）カッコ点・その他'] = df['（3）カッコ点・その他'].fillna(0)
df['（3）ポイント数合計'] = df['（3）ポイント数合計'].fillna(0)
df['（3）配点'] = df['（3）配点'].fillna(0)
df['模範解答の記述行数（３）'] = df['模範解答の記述行数（３）'].fillna(0)


#補足資料の枚数列の例外を修正
df['補足資料の枚数'] = df['補足資料の枚数'].replace('補足なし',0)
df['補足資料の枚数'] = df['補足資料の枚数'].replace('補足基準発行後',np.nan)


df['科目'] = df['科目'].replace('数学X/A\u3000','数学X/A')#0905に修正
#最終ペースを空欄に
df['最終ペース'] = df['最終ペース'].replace('採点回終了時',np.nan)
#在宅勤務フラグ作成
df = make_rank(df,'年度')

#分野名を分野マスタで変換
df = df.rename(columns={'分野':'分野修正前'})
df1 = df
df2 = bunya
df = pandas_tool.merge(df1,df2, key = ["分野修正前"],how = 'left')
#不要な列を削除
df = df.drop(['分野（詳細）', '他参照有無', '分野修正前','Unnamed: 39'], axis=1)
df_math_bunya = df.copy()

df1のキー重複
552
df2のキー重複
0
df1のオブザベーション:584
df2のオブザベーション:159
mergeのオブザベーション:584


In [20]:
"""
書き出し
"""

name = "FMT修正版_CRLEAデータ①入力様式_v2_1222_数学分野修正_v1"
path_folder = r"/Users/s.ogura/Documents/CRLEA/data/intermediatedata"#フォルダパス

df1 = df_math_bunya

#csv
df1.to_csv('{}/{}.csv'.format(path_folder,"{}_{}".format(name,'math')),encoding='utf-8-sig',index=False)